In [ ]:
import cv2
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
from PIL import Image
import os
from database2 import DehazingDataset
import matplotlib.pyplot as plt
import random

# Functions for augmenting hazy images

In [ ]:
def add_haze(image):
    # Convert image to uint8 data type
    image_uint8 = (image * 255).astype(np.uint8)
    
    # Simulate haze by blending the image with a white overlay
    overlay = np.full_like(image_uint8, (255, 255, 255), dtype=np.uint8)  # White overlay
    alpha = random.uniform(0.1, 0.5)  # Random transparency level
    haze_image = cv2.addWeighted(image_uint8, 1 - alpha, overlay, alpha, 0)
    
    return haze_image

def add_haze_gaussian(image):
    # Apply Gaussian blur to the image
    blurred_image = cv2.GaussianBlur(image, (15, 15), 0)
    
    # Simulate haze by blending the original image with the blurred image
    alpha = random.uniform(0.1, 0.5)  # Random transparency level
    haze_image = cv2.addWeighted(image, 1 - alpha, blurred_image, alpha, 0)
    return haze_image

def add_noise(image):
    # Generate Gaussian noise
    mean = 0
    sigma = random.uniform(10, 30)  # Random standard deviation for noise
    noise = np.random.normal(mean, sigma, image.shape).astype(image.dtype)  # Ensure same data type as image
    
    # Add noise to the image
    noisy_image = cv2.add(image, noise)
    return noisy_image

def add_atmospheric_scattering(image, depth_map=None):
    # Simulate atmospheric scattering by attenuating light based on depth
    if depth_map is None:
        depth_map = np.ones_like(image, dtype=np.float32)  # Default to uniform depth map
    
    # Define parameters for scattering
    scattering_coefficient = random.uniform(0.1, 0.2)  # Random scattering coefficient
    light_intensity = random.uniform(1, 3.0)  # Random light intensity
    
    # Simulate scattering effect
    scattered_image = image * np.exp(-scattering_coefficient * depth_map * light_intensity)
    
    return scattered_image

def augment_hazy_image(image):
    # Apply a combination of augmentation techniques
    augmented_image = add_haze(image)
    augmented_image = add_haze_gaussian(augmented_image)
    augmented_image = add_noise(augmented_image)
    augmented_image = add_atmospheric_scattering(augmented_image)
    
    return augmented_image


# Add haze & random occlusion are useful

In [ ]:
def add_haze(image):
    # Convert image to uint8 data type
    image_uint8 = (image * 255).astype(np.uint8)
    
    # Simulate haze by blending the image with a white overlay
    overlay = np.full_like(image_uint8, (255, 255, 255), dtype=np.uint8)  # White overlay
    alpha = random.uniform(0.1, 0.5)  # Random transparency level
    haze_image = cv2.addWeighted(image_uint8, 1 - alpha, overlay, alpha, 0)
    
    return haze_image


def add_random_occlusion(image):
    # Randomly generate occlusions (black patches) in the image
    occlusion_mask = np.random.choice([0, 1], size=image.shape[:2], p=[0.45, 0.55])  # 5% chance of occlusion
    occluded_image = image.copy()
    occluded_image[occlusion_mask.astype(bool)] = 0
    return occluded_image


def apply_gradient_overlay(image):
    # Generate a linear gradient overlay
    gradient = np.linspace(0, 1, image.shape[0])
    gradient_overlay = (gradient[:, np.newaxis] * np.ones((image.shape[1], 3))).astype(np.uint8)
    
    # Resize the gradient overlay to match the size of the input image
    gradient_overlay_resized = cv2.resize(gradient_overlay, (image.shape[1], image.shape[0]))
    
    # Check the dimensions of both images before blending
    print("Image shape:", image.shape)
    print("Gradient overlay shape:", gradient_overlay_resized.shape)
    
    return cv2.addWeighted(image, 0.5, gradient_overlay_resized, 0.5, 0)

def add_random_lighting(image):
    # Simulate random lighting effects
    brightness = random.uniform(0.5, 1.5)  # Random brightness adjustment
    contrast = random.uniform(0.5, 1.5)  # Random contrast adjustment
    lightened_image = cv2.convertScaleAbs(image, alpha=contrast, beta=brightness)
    return lightened_image

def apply_color_distortion(image):
    # Simulate color distortion or tinting
    red_scale = random.uniform(0.8, 1.2)  # Random scaling factor for red channel
    green_scale = random.uniform(0.8, 1.2)  # Random scaling factor for green channel
    blue_scale = random.uniform(0.8, 1.2)  # Random scaling factor for blue channel
    distorted_image = image * np.array([blue_scale, green_scale, red_scale])
    return np.clip(distorted_image, 0, 255).astype(np.uint8)

def apply_motion_blur(image):
    # Simulate motion blur
    kernel_size = random.choice([3, 5, 7])  # Random kernel size for blur
    motion_blur_kernel = np.zeros((kernel_size, kernel_size))
    motion_blur_kernel[int((kernel_size - 1) / 2), :] = np.ones(kernel_size)
    motion_blur_kernel /= kernel_size
    return cv2.filter2D(image, -1, motion_blur_kernel)


def add_random_black_clouds(image):
    # Randomly generate black clouds (dark patches) in the image
    cloud_mask = np.random.choice([0, 1], size=image.shape[:2], p=[0.2, 0.8])  # 20% chance of cloud
    cloud_image = image.copy()
    cloud_image[cloud_mask.astype(bool)] = 0
    return cloud_image

def augment_hazy_image(image):
    # Apply a combination of augmentation techniques
    augmented_image = add_haze(image)
    augmented_image = add_random_occlusion(augmented_image)
    augmented_image = apply_gradient_overlay(augmented_image)
    augmented_image = add_random_lighting(augmented_image)
    augmented_image = apply_color_distortion(augmented_image)
    augmented_image = apply_motion_blur(augmented_image)
    return augmented_image


# Functions from https://www.freecodecamp.org/news/image-augmentation-make-it-rain-make-it-snow-how-to-modify-a-photo-with-machine-learning-163c0cb3843f/

In [ ]:
def add_brightness(image):
    image_HLS = cv2.cvtColor(image,cv2.COLOR_RGB2HLS)
    ## Conversion to HLS
    image_HLS = np.array(image_HLS, dtype = np.float64)
    random_brightness_coefficient = np.random.uniform()+0.5 ## generates value between 0.5 and 1.5
    image_HLS[:,:,1] = image_HLS[:,:,1]*random_brightness_coefficient ## scale pixel values up or down for channel 1(Lightness)
    image_HLS[:,:,1][image_HLS[:,:,1]>255]  = 255 ##Sets all values above 255 to 255
    image_HLS = np.array(image_HLS, dtype = np.uint8)
    image_RGB = cv2.cvtColor(image_HLS,cv2.COLOR_HLS2RGB)
    ## Conversion to RGB
    return image_RGB


import numpy as np

def generate_shadow_coordinates(imshape, no_of_shadows=1):
    vertices_list = []
    for index in range(no_of_shadows):
        vertex = []
        for dimensions in range(np.random.randint(3, 15)):  # Dimensionality of the shadow polygon
            vertex.append((imshape[1] * np.random.uniform(), imshape[0] // 3 + imshape[0] * np.random.uniform()))
        vertices = np.array([vertex], dtype=np.int32)  # Single shadow vertices
        vertices_list.append(vertices)
    return vertices_list  # List of shadow vertices


import cv2
import numpy as np

def add_shadow(image, no_of_shadows=1):
    image_HLS = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)  # Conversion to HLS
    mask = np.zeros_like(image)
    imshape = image.shape
    vertices_list = generate_shadow_coordinates(imshape, no_of_shadows)  # Getting list of shadow vertices
    
    for vertices in vertices_list:
        cv2.fillPoly(mask, vertices, 255)  # Adding all shadow polygons on empty mask, single 255 denotes only red channel
        image_HLS[:,:,1][mask[:,:,0] == 255] = image_HLS[:,:,1][mask[:,:,0] == 255] * 0.5  # If red channel is hot, image's "Lightness" channel's brightness is lowered
    
    image_RGB = cv2.cvtColor(image_HLS, cv2.COLOR_HLS2RGB)  # Conversion to RGB
    return image_RGB

import cv2
import numpy as np

def add_snow(image):
    image_HLS = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)  # Conversion to HLS
    image_HLS = np.array(image_HLS, dtype=np.float64)
    brightness_coefficient = 2.5
    snow_point = 140  # Increase this for more snow
    
    # Scale pixel values up for channel 1 (Lightness)
    image_HLS[:,:,1][image_HLS[:,:,1] < snow_point] = image_HLS[:,:,1][image_HLS[:,:,1] < snow_point] * brightness_coefficient
    
    # Set all values above 255 to 255
    image_HLS[:,:,1][image_HLS[:,:,1] > 255] = 255
    
    image_HLS = np.array(image_HLS, dtype=np.uint8)
    image_RGB = cv2.cvtColor(image_HLS, cv2.COLOR_HLS2RGB)  # Conversion to RGB
    return image_RGB
import cv2
import numpy as np

def generate_random_lines(imshape, slant, drop_length):
    drops = []
    for i in range(1500):  # If you want heavy rain, try increasing this
        if slant < 0:
            x = np.random.randint(slant, imshape[1])
        else:
            x = np.random.randint(0, imshape[1] - slant)
        y = np.random.randint(0, imshape[0] - drop_length)
        drops.append((x, y))
    return drops

def add_rain(image):
    imshape = image.shape
    slant_extreme = 10
    slant = np.random.randint(-slant_extreme, slant_extreme)
    drop_length = 20
    drop_width = 2
    drop_color = (200, 200, 200)  # A shade of gray
    rain_drops = generate_random_lines(imshape, slant, drop_length)
    
    for rain_drop in rain_drops:
        cv2.line(image, (rain_drop[0], rain_drop[1]), (rain_drop[0] + slant, rain_drop[1] + drop_length), drop_color, drop_width)
    
    image = cv2.blur(image, (7, 7))  # Rainy views are blurry
    
    brightness_coefficient = 0.7  # Rainy days are usually shady
    image_HLS = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)  # Conversion to HLS
    image_HLS[:,:,1] = image_HLS[:,:,1] * brightness_coefficient  # Scale pixel values down for channel 1 (Lightness)
    image_RGB = cv2.cvtColor(image_HLS, cv2.COLOR_HLS2RGB)  # Conversion to RGB
    
    return image_RGB
import cv2
import numpy as np

def add_blur(image, x, y, hw):
    image[y:y+hw, x:x+hw, 1] = image[y:y+hw, x:x+hw, 1] + 1
    image[:,:,1][image[:,:,1] > 255] = 255  # Sets all values above 255 to 255
    image[y:y+hw, x:x+hw, 1] = cv2.blur(image[y:y+hw, x:x+hw, 1], (10, 10))
    return image

def generate_random_blur_coordinates(imshape, hw):
    blur_points = []
    midx = imshape[1] // 2 - hw - 100
    midy = imshape[0] // 2 - hw - 100
    index = 1
    while midx > -100 or midy > -100:  # Radially generating coordinates
        for i in range(250 * index):
            x = np.random.randint(midx, imshape[1] - midx - hw)
            y = np.random.randint(midy, imshape[0] - midy - hw)
            blur_points.append((x, y))
        midx -= 250 * imshape[1] // sum(imshape)
        midy -= 250 * imshape[0] // sum(imshape)
        index += 1
    return blur_points

def add_fog(image):
    image_HLS = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)  # Conversion to HLS
    mask = np.zeros_like(image)
    imshape = image.shape
    hw = 100
    image_HLS[:,:,1] = image_HLS[:,:,1] * 0.8
    haze_list = generate_random_blur_coordinates(imshape, hw)
    
    for haze_points in haze_list:
        image_HLS[:,:,1][image_HLS[:,:,1] > 255] = 255  # Sets all values above 255 to 255
        image_HLS = add_blur(image_HLS, haze_points[0], haze_points[1], hw)
    
    image_RGB = cv2.cvtColor(image_HLS, cv2.COLOR_HLS2RGB)  # Conversion to RGB
    return image_RGB




In [ ]:
# Directory containing your dataset
dataset_dir = "Task2Dataset"


In [ ]:
root_dir = 'Task2Dataset'
train_dir = os.path.join(root_dir, 'train')
val_dir = os.path.join(root_dir, 'val')
transform = transforms.Compose([
                                #  transforms.Resize((224, 224)), # ASSUMING NO NEED FOR RESIZING AS ALL IMAGES ARE ALREADY 256*256
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
                                 ])

train_dataset = DehazingDataset(train_dir, transform)
val_dataset = DehazingDataset(val_dir, transform)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

# Viewing clean and corresponding augmented images 

In [ ]:

def show_images(dataloader, num_images=5):
    # Get a batch of data
    data_iter = iter(dataloader)
    _, images = next(data_iter)
    
    # Plot original clean images
    fig, axes = plt.subplots(6, num_images, figsize=(15, 15))
    for i in range(num_images):
        clean_image = images[i].permute(1, 2, 0).cpu().numpy()  # Convert to NumPy array
        clean_image = clean_image * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])  # Denormalize
        
        axes[0, i].imshow(clean_image)
        axes[0, i].axis('off')
        axes[0, i].set_title("Clean Image")
        
        # Apply each augmentation function separately and plot the result
        augmented_haze_image = add_haze(clean_image)
        axes[1, i].imshow(augmented_haze_image)
        axes[1, i].axis('off')
        axes[1, i].set_title("Haze Only")
        
        augmented_gaussian_haze_image = add_haze_gaussian(clean_image)
        axes[2, i].imshow(augmented_gaussian_haze_image)
        axes[2, i].axis('off')
        axes[2, i].set_title("Haze with Gaussian Blur")
        
        noisy_image = add_noise(clean_image)
        axes[3, i].imshow(noisy_image)
        axes[3, i].axis('off')
        axes[3, i].set_title("Noisy Image")
        
        atmospheric_scattering_image = add_atmospheric_scattering(clean_image)
        axes[4, i].imshow(atmospheric_scattering_image)
        axes[4, i].axis('off')
        axes[4, i].set_title("Atmospheric Scattering")
        
        # Apply all augmentation functions together
        all_augmented_image = augment_hazy_image(clean_image)
        axes[5, i].imshow(all_augmented_image)
        axes[5, i].axis('off')
        axes[5, i].set_title("All Augmentations Combined")
    
    plt.tight_layout()
    plt.show()

# Visualize images from train dataloader
print("Images from Train Dataloader:")
show_images(train_dataloader)

# Visualize images from validation dataloader
print("Images from Validation Dataloader:")
show_images(val_dataloader)

In [ ]:
def show_images(dataloader, num_images=5):
    # Get a batch of data
    data_iter = iter(dataloader)
    _, images = next(data_iter)
    
    # Plot original clean images
    fig, axes = plt.subplots(6, num_images, figsize=(15, 15))
    for i in range(num_images):
        clean_image = images[i].permute(1, 2, 0).cpu().numpy()  # Convert to NumPy array
        clean_image = clean_image * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])  # Denormalize
        
        
        axes[0, i].imshow(clean_image)
        axes[0, i].axis('off')
        axes[0, i].set_title("Clean Image")
        
        # Apply each augmentation function separately and plot the result
        augmented_haze_image = add_haze(clean_image)
        axes[1, i].imshow(augmented_haze_image)
        axes[1, i].axis('off')
        axes[1, i].set_title("Haze Only")
        
        random_occlusion_image = add_random_occlusion(clean_image)
        axes[2, i].imshow(random_occlusion_image)
        axes[2, i].axis('off')
        axes[2, i].set_title("Random Occlusion")
        
        gradient_overlay_image = add_random_black_clouds(clean_image)
        axes[3, i].imshow(gradient_overlay_image)
        axes[3, i].axis('off')
        axes[3, i].set_title("Black Cloud")
        
        random_lighting_image = add_random_lighting(clean_image)
        axes[4, i].imshow(random_lighting_image)
        axes[4, i].axis('off')
        axes[4, i].set_title("Random Lighting")
        
        color_distorted_image = apply_color_distortion(clean_image)
        axes[5, i].imshow(color_distorted_image)
        axes[5, i].axis('off')
        axes[5, i].set_title("Color Distortion")
    
    plt.tight_layout()
    plt.show()

# Visualize images from train dataloader
print("Images from Train Dataloader:")
show_images(train_dataloader)

# Visualize images from validation dataloader
print("Images from Validation Dataloader:")
show_images(val_dataloader)


# Viewing clean and corresponding hazy images from existing dataset

In [ ]:
def show_images(dataloader, num_images=5):
    # Get a batch of data
    data_iter = iter(dataloader)
    hazy_images, clean_images = next(data_iter)
    
    # Denormalize images
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    
    clean_images = clean_images.numpy().transpose((0, 2, 3, 1))
    clean_images = std * clean_images + mean
    clean_images = np.clip(clean_images, 0, 1)
    
    hazy_images = hazy_images.numpy().transpose((0, 2, 3, 1))
    hazy_images = std * hazy_images + mean
    hazy_images = np.clip(hazy_images, 0, 1)
    
    # Plot images
    fig, axes = plt.subplots(num_images, 2, figsize=(15, num_images * 3))
    for i in range(num_images):
        axes[i, 0].imshow(clean_images[i])
        axes[i, 0].set_title("Clean Image")
        axes[i, 0].axis('off')
        
        axes[i, 1].imshow(hazy_images[i])
        axes[i, 1].set_title("Hazy Image")
        axes[i, 1].axis('off')
        
    plt.tight_layout()
    plt.show()

# Visualize images from train dataloader
print("Images from Train Dataloader:")
show_images(train_dataloader)

# Visualize images from validation dataloader
print("Images from Validation Dataloader:")
show_images(val_dataloader)
